In [ ]:
using Pkg
pkg"add MeshCat"
pkg"add MeshCatMechanisms"
pkg"add RigidBodySim"
pkg"add RigidBodyDynamics"
pkg"add StaticArrays"

In [ ]:
using MeshCatMechanisms
using RigidBodyDynamics
using RigidBodySim
using MeshCat
using StaticArrays

In [ ]:
vis = Visualizer()
open(vis)

In [ ]:
basename="Stomp"
urdf = basename * ".urdf"
run(`/bin/bash -c '. /opt/ros/melodic/setup.bash \&\& xacro --inorder --check-order -o '$(urdf)' '$(basename)'.xacro'`)
stomp = parse_urdf(urdf)
remove_fixed_tree_joints!(stomp);
delete!(vis)
mvis = MechanismVisualizer(stomp, URDFVisuals(urdf), vis)

In [ ]:
state = MechanismState(stomp)
open_loop_dynamics = Dynamics(stomp)

In [ ]:
final_time = 10.
problem = ODEProblem(open_loop_dynamics, state, (0., final_time))

In [ ]:
gui_callback = CallbackSet(mvis)
rate_limiter = RealtimeRateLimiter(max_rate = 1.)
callbacks = CallbackSet(gui_callback,rate_limiter)
sol = solve(problem, Tsit5(), abs_tol=1e-10, dt=0.05, callback=callbacks)